Modes of correlation: Model (oracle, personal), Granularity (recipe level, user level, recipe flat, user flat, all recipes for all users), Surprise (95th percentile, max), Method of correlation (spearman, pearson)

Libraries

In [1]:
import os, pickle, numpy as np
from GloVex.evaluate_personalised import survey_reader
from scipy.stats import pearsonr, spearmanr
cwd = os.getcwd()

Helper functions

In [2]:
def averageing_fn(surp_list):
	_avg_ = []
	for each_ in surp_list:
		_avg_.append(reduce(lambda x, y: x + y, each_) / len(each_))
	return _avg_

Get the surprises of the users in the survey

In [3]:
survey_fp = '/Volumes/GoogleDrive/My Drive/Live/PQE/Omar/personalized-surprise/qchef_surveydata'
user_fam_dict, users_surp_ratings_dict = survey_reader(survey_fp)

GloVex/evaluate_personalised.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  users_surp_ratings_df['surp_list'] = users_surp_ratings_df.values.tolist()


Prepare array from the dict

In [4]:
# Take the familiarity scores from the dict into arrays
user_fam_arr = []
for each_user in user_fam_dict:
	user_fam_arr.append(user_fam_dict[each_user])
# Take the surprise scores from the dict into arrays
users_surp_ratings_arr = []
for each_user in users_surp_ratings_dict:
	users_surp_ratings_arr.append(users_surp_ratings_dict[each_user])

Transpose for recipe level comparison

In [5]:
users_surp_ratings_userlvl = users_surp_ratings_arr
users_surp_ratings_recipelvl = np.transpose(users_surp_ratings_arr).tolist()

Flatten the arrays in both dimenssions

In [6]:
users_surp_ratings_user_flat = [item for sublist in users_surp_ratings_userlvl for item in sublist]
users_surp_ratings_recipe_flat = [item for sublist in users_surp_ratings_recipelvl for item in sublist]

Divide the users into foodies and novices using the median of sum of familiarities

In [7]:
# Array holding all of the sums
sum_arr = []
# Iterate over the users' familiarities
for user_idx in user_fam_dict.keys():
	# Get the familiarity scores
	each_fam = user_fam_dict[user_idx]
	# Calculate the sum of familiarity for each user
	sum_fam = sum(each_fam)
	# Append the sum of familiarity of each user to the sum array
	sum_arr.append(sum_fam)
median_fam = np.median(sum_arr)
# Split the users into foodeis and novices using the median
foodie_fam_dict = {}
foodie_surp_ratings_dict = {}
novice_fam_dict = {}
novice_surp_ratings_dict = {}
# Iterate over the users' familiarities
for user_idx in user_fam_dict.keys():
	# Get the familiarity and surprise scores
	each_fam = user_fam_dict[user_idx]
	each_surp = users_surp_ratings_dict[user_idx]
	# Calculate the sum of familiarity for each user
	sum_fam = sum(each_fam)
	if sum_fam > median_fam:
		# print 'above median', each_fam
		foodie_fam_dict[user_idx] = each_fam
		foodie_surp_ratings_dict[user_idx] = each_surp
	else:
		# print 'below median', each_fam
		novice_fam_dict[user_idx] = each_fam
		novice_surp_ratings_dict[user_idx] = each_surp
# List the foodies and novices in two separate lists
foodie_users = foodie_fam_dict.keys()
novice_users = novice_fam_dict.keys()
# print 'foodie_fam_dict', foodie_fam_dict
# print 'foodie_surp_ratings_dict', foodie_surp_ratings_dict
# print 'novice_fam_dict', novice_fam_dict
# print 'novice_surp_ratings_dict', novice_surp_ratings_dict

Get the personal surprise estimates by the model

In [8]:
user_suprise_estimates_pickle_fn = cwd + '/GloVex/results/new/user_suprise_estimates.pickle'
user_suprise_estimates_dict = pickle.load(open(user_suprise_estimates_pickle_fn,'rb'))

Get the 95th percentils and max surprise estimates

In [9]:
# Initialize surprise estimates for 95th percentile and max score
personalized_surp_estimates_95perc = []
personalized_surp_estimates_95perc_dict = {}
personalized_surp_estimates_max = []
personalized_surp_estimates_max_dict = {}
# Iterate over the user surprise estimeates dict
for each_user in user_suprise_estimates_dict.keys():
	# print 'each_user', each_user
	# print user_suprise_estimates_dict[each_user]
	# Initialize the inner arrays for 95th percentile and max score
	users_surp_estimates_95perc = []
	users_surp_estimates_max = []
	# Iterate over the surprises for each user's surprise estimates
	for recipe_idx, each_recipe in enumerate(user_suprise_estimates_dict[each_user]['recipes_surp']):
		# print 'recipe_idx', recipe_idx, user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['95th_percentile']
		# Get all of the ingredients combinations' surprise estimates
		ingr_comb_suprise_arr = [each_ingr_comb[2] for each_ingr_comb in 
			user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['surprise_cuisine']]
		# Get the 95th percentile
		user_suprise_95th_percentile = user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['95th_percentile']
		# print user_suprise_95th_percentile
		# Get the max
		user_suprise_max = max(ingr_comb_suprise_arr)
		# print user_suprise_max
		# Append the 95th percentile for the surprise estimates
		users_surp_estimates_95perc.append(user_suprise_95th_percentile)
		# Append the max score for the surprise estimates
		users_surp_estimates_max.append(user_suprise_max)
	# Append the user's surprise estimates to the personalized main 2D array (95th percentile and max score)
	personalized_surp_estimates_95perc.append(users_surp_estimates_95perc)
	personalized_surp_estimates_max.append(users_surp_estimates_max)
	# 
	personalized_surp_estimates_95perc_dict[each_user] = users_surp_estimates_95perc
	personalized_surp_estimates_max_dict[each_user] = users_surp_estimates_max
# print personalized_surp_estimates_95perc
# print personalized_surp_estimates_max
# print personalized_surp_estimates_95perc_dict
# print personalized_surp_estimates_max_dict

Transpose for recipe level comparison

In [10]:
personalized_surp_estimates_recipelvl_95perc = np.transpose(personalized_surp_estimates_95perc).tolist()
personalized_surp_estimates_recipelvl_max = np.transpose(personalized_surp_estimates_max).tolist()

Flatten the arrays in both dimnessions

In [11]:
personalized_surp_estimates_userflat_95perc = [item for sublist in personalized_surp_estimates_95perc for item in sublist]
personalized_surp_estimates_userflat_max = [item for sublist in personalized_surp_estimates_max for item in sublist]
personalized_surp_estimates_recipeflat_95perc = [item for sublist in personalized_surp_estimates_recipelvl_95perc for item in sublist]
personalized_surp_estimates_recipeflat_max = [item for sublist in personalized_surp_estimates_recipelvl_max for item in sublist]

Separate the foodies from novices

In [12]:
# Get the foodies for the 95th percentile and the max
foodie_personalized_surp_estimates_95perc_dict = {each_: personalized_surp_estimates_95perc_dict[each_] 
	for each_ in personalized_surp_estimates_95perc_dict.keys() if each_ in foodie_surp_ratings_dict.keys()}
foodie_personalized_surp_estimates_max_dict = {each_: personalized_surp_estimates_max_dict[each_] 
	for each_ in personalized_surp_estimates_max_dict.keys() if each_ in foodie_surp_ratings_dict.keys()}
# Get the novices for the 95th percentile and the max
novice_personalized_surp_estimates_95perc_dict = {each_: personalized_surp_estimates_95perc_dict[each_] 
	for each_ in personalized_surp_estimates_95perc_dict.keys() if each_ in novice_surp_ratings_dict.keys()}
novice_personalized_surp_estimates_max_dict = {each_: personalized_surp_estimates_max_dict[each_] 
	for each_ in personalized_surp_estimates_max_dict.keys() if each_ in novice_surp_ratings_dict.keys()}

Transfer the foodies' and novices' into arrays using the same iterator's index for the ratings and estimates (at the same time to geuarantee synchronization)

In [13]:
# Foodies
foodie_surp_ratings = []
foodie_personalized_surp_estimates_95perc = []
foodie_personalized_surp_estimates_max = []
for each_foodie in foodie_users:
	foodie_surp_ratings.append(foodie_surp_ratings_dict[each_foodie])
	foodie_personalized_surp_estimates_95perc.append(foodie_personalized_surp_estimates_95perc_dict[each_foodie])
	foodie_personalized_surp_estimates_max.append(foodie_personalized_surp_estimates_max_dict[each_foodie])
# Novices
novice_surp_ratings = []
novice_personalized_surp_estimates_95perc = []
novice_personalized_surp_estimates_max = []
for each_novice in novice_users:
	novice_surp_ratings.append(novice_surp_ratings_dict[each_novice])
	novice_personalized_surp_estimates_95perc.append(novice_personalized_surp_estimates_95perc_dict[each_novice])
	novice_personalized_surp_estimates_max.append(novice_personalized_surp_estimates_max_dict[each_novice])

Foodies' and novices' user and recipe levels

In [14]:
# 
foodie_surp_ratings_userlvl = foodie_surp_ratings
foodie_surp_ratings_recipelvl = np.transpose(foodie_surp_ratings_userlvl).tolist()
# 
foodie_surp_ratings_user_flat = [item for sublist in foodie_surp_ratings_userlvl for item in sublist]
foodie_surp_ratings_recipe_flat = [item for sublist in foodie_surp_ratings_recipelvl for item in sublist]
# 
novice_surp_ratings_userlvl = novice_surp_ratings
novice_surp_ratings_recipelvl = np.transpose(novice_surp_ratings_userlvl).tolist()
# 
novice_surp_ratings_user_flat = [item for sublist in novice_surp_ratings_userlvl for item in sublist]
novice_surp_ratings_recipe_flat = [item for sublist in novice_surp_ratings_recipelvl for item in sublist]

Flatten the foodies' and novices' surprise ratings

In [15]:
# 
foodie_personalized_surp_estimates_userflat_95perc = [item for sublist in foodie_personalized_surp_estimates_95perc for item in sublist]
foodie_personalized_surp_estimates_userflat_max = [item for sublist in foodie_personalized_surp_estimates_max for item in sublist]
# 
foodie_personalized_surp_estimates_recipelvl_95perc = np.transpose(foodie_personalized_surp_estimates_95perc).tolist()
foodie_personalized_surp_estimates_recipeflat_95perc = [item for sublist in foodie_personalized_surp_estimates_recipelvl_95perc for item in sublist]
foodie_personalized_surp_estimates_recipelvl_max = np.transpose(foodie_personalized_surp_estimates_max).tolist()
foodie_personalized_surp_estimates_recipeflat_max = [item for sublist in foodie_personalized_surp_estimates_recipelvl_max for item in sublist]
# 
novice_personalized_surp_estimates_userflat_95perc = [item for sublist in novice_personalized_surp_estimates_95perc for item in sublist]
novice_personalized_surp_estimates_userflat_max = [item for sublist in novice_personalized_surp_estimates_max for item in sublist]
# 
novice_personalized_surp_estimates_recipelvl_95perc = np.transpose(novice_personalized_surp_estimates_95perc).tolist()
novice_personalized_surp_estimates_recipeflat_95perc = [item for sublist in novice_personalized_surp_estimates_recipelvl_95perc for item in sublist]
novice_personalized_surp_estimates_recipelvl_max = np.transpose(novice_personalized_surp_estimates_max).tolist()
novice_personalized_surp_estimates_recipeflat_max = [item for sublist in novice_personalized_surp_estimates_recipelvl_max for item in sublist]

Get the oracle surprise estimates by the model

In [16]:
oracle_suprise_estimates_pickle_fn = cwd + '/GloVex/results/new/oracle_suprise_estimates.pickle'
oracle_suprise_estimates_dict = pickle.load(open(oracle_suprise_estimates_pickle_fn, 'rb'))

Get the oracle's surprise estimates (95th percentile and max score)

In [17]:
# Initialize arrays
oracle_surp_estimates_95perc = []
oracle_surp_estimates_max = []
# Iterate over the dict
for each_recipe in oracle_suprise_estimates_dict:
	# Get all of the ingredients combinations' surprise estimates
	oracle_ingr_surp_estimates = [each_ingr_comb[2] for each_ingr_comb in oracle_suprise_estimates_dict[each_recipe]['surprise_cuisine']]
	# Append the 95th_percentile surprise estimate
	oracle_surp_estimates_95perc.append(oracle_suprise_estimates_dict[each_recipe]['95th_percentile'])
	# Append the max score surprise estimate
	oracle_surp_estimates_max.append(max(oracle_ingr_surp_estimates))
# oracle_surp_estimates_95perc
# oracle_surp_estimates_max

Averages

In [18]:
# All users ratings
users_surp_ratings_recipelvl_avg = averageing_fn(users_surp_ratings_recipelvl)
# Foodies ratings
foodie_surp_ratings_recipelvl_avg = averageing_fn(foodie_surp_ratings_recipelvl)
# Novices ratings
novice_surp_ratings_recipelvl_avg = averageing_fn(novice_surp_ratings_recipelvl)
# All users estimates 95th perc
personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(personalized_surp_estimates_recipelvl_95perc)
# All users estimates max
personalized_surp_estimates_recipelvl_max_avg = averageing_fn(personalized_surp_estimates_recipelvl_max)
# Foodies estimates 95th perc
foodie_personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(foodie_personalized_surp_estimates_recipelvl_95perc)
# Foodies estimates max
foodie_personalized_surp_estimates_recipelvl_max_avg = averageing_fn(foodie_personalized_surp_estimates_recipelvl_max)
# Novices estimates 95th perc
novice_personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(novice_personalized_surp_estimates_recipelvl_95perc)
# Novices estimates max
novice_personalized_surp_estimates_recipelvl_max_avg = averageing_fn(novice_personalized_surp_estimates_recipelvl_max)

Find correlation between surprise scores

Function for calculating correlation per user

In [19]:
# Function for calculating the correlation per user
def corr_per_user(oracle_surp_estimates, personalized_surp_estimates, users_surp_ratings_userlvl):
	for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates, users_surp_ratings_userlvl)):
		print user_idx
		print each_personal_score
		print each_users_rating
		print 'oracle_surp_estimates: spearmanr', spearmanr(oracle_surp_estimates, each_users_rating)
		print 'oracle_surp_estimates: pearsonr', pearsonr(oracle_surp_estimates, each_users_rating)
		print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
		print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

Mode: Oracle + Personalized, per user (default for oracle), 95th percentile and max

In [20]:
corr_per_user(oracle_surp_estimates_95perc, personalized_surp_estimates_95perc, users_surp_ratings_userlvl)

0
[0.5825177510358344, -0.1883569539730416, 1.355178341548168, 0.20608820945069306, 1.467778269141499, 1.3938414757134172, 0.7591681821732095, 2.6606224070108926, 3.935509136160972, 0.11588770365483453]
[1, 1, 4, 4, 1, 4, 5, 4, 3, 4]
oracle_surp_estimates: spearmanr (0.30813875924572515, 0.38638381485150064)
oracle_surp_estimates: pearsonr (0.22543514207124077, 0.5311675107950075)
each_personal_score: spearmanr (0.03278071906869416, 0.9283691828415599)
each_personal_score: pearsonr (0.13238444175275613, 0.7154311551200501)
1
[0.3766410246138465, -0.47380563222987676, 1.1411009226142137, -0.04980279414294565, 1.197845697498431, 1.448952820949485, 0.5170783892044251, 2.0425914469623736, 3.3427475369670065, -0.15858495170931128]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
oracle_surp_estimates: spearmanr (nan, nan)
oracle_surp_estimates: pearsonr (nan, 1.0)
each_personal_score: spearmanr (nan, nan)
each_personal_score: pearsonr (nan, 1.0)
2
[0.4655375735178752, -0.40885300118868806, 1.2049804637360655

/Users/oeltayeb/Library/Python/2.7/lib/python/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/oeltayeb/Library/Python/2.7/lib/python/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/scipy/stats/distributions.py:1415: RuntimeWarning: invalid value encountered in greater
  cond1 = (scale > 0) & (x > self.a) & (x < self.b)
/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/scipy/stats/distributions.py:1415: RuntimeWarning: invalid value encountered in less
  cond1 = (scale > 0) & (x > self.a) & (x < self.b)
/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/scipy/stats/distributions.py:1416: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
/System/Library/Frameworks/Py

 (0.3901845186230132, 0.26497624660549096)
each_personal_score: pearsonr (0.32999203478476163, 0.35174617882857967)
46
[0.027091592501152915, -0.8488955492842698, 0.6673493758531812, -0.40989073073075305, 0.7952293321101441, 1.1346956112300324, 0.16183047751462978, 1.6125765575012398, 2.9837052568300417, -0.49611307294316054]
[1, 1, 3, 4, 4, 3, 3, 4, 4, 3]
oracle_surp_estimates: spearmanr (0.18164975363768865, 0.6154951256738584)
oracle_surp_estimates: pearsonr (0.2661734635795347, 0.45727330429710594)
each_personal_score: spearmanr (0.5968491905238341, 0.06851084629788723)
each_personal_score: pearsonr (0.5552835266187389, 0.09564877810974974)
47
[0.5762016430116379, -0.19675566414633752, 1.354640507685319, 0.21251095333620598, 1.4421512434211774, 1.4004048942717855, 0.7627085151965424, 2.596251562006935, 3.95349188672306, 0.12214063961289182]
[1, 1, 1, 2, 4, 1, 2, 1, 1, 1]
oracle_surp_estimates: spearmanr (0.6741998624632421, 0.03251559932021606)
oracle_surp_estimates: pearsonr (0.47

In [21]:
corr_per_user(oracle_surp_estimates_max, personalized_surp_estimates_max, users_surp_ratings_userlvl)

0
[1.0420700709383728, 0.030664403631934244, 1.6407100585617616, 0.5552353074288633, 4.670522777983191, 2.153884866828632, 1.3698882649964497, 2.75698945026687, 4.531743960318245, 1.05329683984565]
[1, 1, 4, 4, 1, 4, 5, 4, 3, 4]
oracle_surp_estimates: spearmanr (0.36058790975563576, 0.3060212830567241)
oracle_surp_estimates: pearsonr (0.2882286617927404, 0.4193198598657479)
each_personal_score: spearmanr (0.006556143813738832, 0.9856590518353308)
each_personal_score: pearsonr (-0.11853928022707874, 0.7443083434726215)
1
[0.8217086324646126, -0.2523357314195088, 1.3192155192712238, 0.3384958204178502, 4.329541864163009, 2.051400678374034, 1.0726043419402524, 2.5874494667619254, 4.576220314765044, 0.4920155954536476]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
oracle_surp_estimates: spearmanr (nan, nan)
oracle_surp_estimates: pearsonr (nan, 1.0)
each_personal_score: spearmanr (nan, nan)
each_personal_score: pearsonr (nan, 1.0)
2
[0.9499809219691805, -0.2203258671490411, 1.2694678168327085, 0.30544103

 (-0.5617690055391233, 0.09102760220353594)
oracle_surp_estimates: pearsonr (-0.6005067497371198, 0.06639782344434898)
each_personal_score: spearmanr (0.11790213696500118, 0.7456444694022333)
each_personal_score: pearsonr (0.11484588789191759, 0.7520620366698232)
50
[0.9017203980908947, -0.20188022036687486, 1.3225983248517144, 0.35236412476590256, 4.368868921684666, 2.107960080675202, 1.1598161125719335, 2.5027878935205923, 4.39544756259255, 0.6299822027957884]
[2, 1, 2, 1, 3, 3, 2, 5, 1, 3]
oracle_surp_estimates: spearmanr (0.18251993707859557, 0.6137747253450212)
oracle_surp_estimates: pearsonr (0.006012104617502934, 0.9868489965044007)
each_personal_score: spearmanr (0.38392124695842517, 0.27339593290149633)
each_personal_score: pearsonr (0.26800323993205777, 0.4540676154316214)
51
[0.5920639866085953, -0.47679565844456867, 1.09059891263544, 0.13694544610817577, 4.052470756591474, 1.6226455365738879, 0.9167753382831787, 2.352658563840303, 4.163235202102575, 0.36222465322349134]
[4,

In [22]:
# 
corr_per_user(oracle_surp_estimates_95perc, foodie_personalized_surp_estimates_95perc, foodie_surp_ratings_userlvl)

0
[0.5825177510358344, -0.1883569539730416, 1.355178341548168, 0.20608820945069306, 1.467778269141499, 1.3938414757134172, 0.7591681821732095, 2.6606224070108926, 3.935509136160972, 0.11588770365483453]
[1, 1, 4, 4, 1, 4, 5, 4, 3, 4]
oracle_surp_estimates: spearmanr (0.30813875924572515, 0.38638381485150064)
oracle_surp_estimates: pearsonr (0.22543514207124077, 0.5311675107950075)
each_personal_score: spearmanr (0.03278071906869416, 0.9283691828415599)
each_personal_score: pearsonr (0.13238444175275613, 0.7154311551200501)
1
[0.4897748608014796, -0.30375615612352524, 1.2834389190905422, 0.11014029115356468, 1.3673984747213206, 1.4375970493359218, 0.6638687865262688, 2.4127100908837447, 3.8074222926611005, 0.020363060304277175]
[1, 1, 1, 2, 2, 1, 1, 3, 2, 1]
oracle_surp_estimates: spearmanr (0.11096671714353053, 0.7602272902237964)
oracle_surp_estimates: pearsonr (0.15778600536212775, 0.6633086826455221)
each_personal_score: spearmanr (0.5340273262532407, 0.11182946483575179)
each_perso

In [23]:
corr_per_user(oracle_surp_estimates_max, foodie_personalized_surp_estimates_max, foodie_surp_ratings_userlvl)

0
[1.0420700709383728, 0.030664403631934244, 1.6407100585617616, 0.5552353074288633, 4.670522777983191, 2.153884866828632, 1.3698882649964497, 2.75698945026687, 4.531743960318245, 1.05329683984565]
[1, 1, 4, 4, 1, 4, 5, 4, 3, 4]
oracle_surp_estimates: spearmanr (0.36058790975563576, 0.3060212830567241)
oracle_surp_estimates: pearsonr (0.2882286617927404, 0.4193198598657479)
each_personal_score: spearmanr (0.006556143813738832, 0.9856590518353308)
each_personal_score: pearsonr (-0.11853928022707874, 0.7443083434726215)
1
[0.9531598300843317, -0.08792162015070339, 1.543062636875322, 0.46334101344641015, 4.54495910956986, 2.0883795640418783, 1.2545495372983018, 2.752803880849236, 4.6757088855527265, 0.8761482201227383]
[1, 1, 1, 2, 2, 1, 1, 3, 2, 1]
oracle_surp_estimates: spearmanr (0.20112717482264905, 0.577404050163908)
oracle_surp_estimates: pearsonr (0.08485019883792277, 0.8157207357072436)
each_personal_score: spearmanr (0.506285646967358, 0.13538537982648902)
each_personal_score: pe

In [24]:
# 
corr_per_user(oracle_surp_estimates_95perc, novice_personalized_surp_estimates_95perc, novice_surp_ratings_userlvl)

0
[0.3766410246138465, -0.47380563222987676, 1.1411009226142137, -0.04980279414294565, 1.197845697498431, 1.448952820949485, 0.5170783892044251, 2.0425914469623736, 3.3427475369670065, -0.15858495170931128]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
oracle_surp_estimates: spearmanr (nan, nan)
oracle_surp_estimates: pearsonr (nan, 1.0)
each_personal_score: spearmanr (nan, nan)
each_personal_score: pearsonr (nan, 1.0)
1
[0.4655375735178752, -0.40885300118868806, 1.2049804637360655, -0.06128611154667095, 1.221876485805888, 1.5757667704471863, 0.6833935832172517, 2.4118355949843435, 3.6042779998780854, 0.05620267139500165]
[2, 2, 5, 2, 5, 2, 2, 2, 1, 5]
oracle_surp_estimates: spearmanr (-0.11096671714353053, 0.7602272902237964)
oracle_surp_estimates: pearsonr (-0.24579384319458147, 0.4936487920767738)
each_personal_score: spearmanr (-0.24967511357294367, 0.48662776459676915)
each_personal_score: pearsonr (-0.27833463302686756, 0.43615920637452205)
2
[-0.1015438469289178, -0.826585756445594, 0.52023148

In [25]:
corr_per_user(oracle_surp_estimates_max, novice_personalized_surp_estimates_max, novice_surp_ratings_userlvl)

0
[0.8217086324646126, -0.2523357314195088, 1.3192155192712238, 0.3384958204178502, 4.329541864163009, 2.051400678374034, 1.0726043419402524, 2.5874494667619254, 4.576220314765044, 0.4920155954536476]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
oracle_surp_estimates: spearmanr (nan, nan)
oracle_surp_estimates: pearsonr (nan, 1.0)
each_personal_score: spearmanr (nan, nan)
each_personal_score: pearsonr (nan, 1.0)
1
[0.9499809219691805, -0.2203258671490411, 1.2694678168327085, 0.3054410318360214, 4.375603272854356, 2.141916109332202, 1.2151057548941362, 2.654977834757437, 4.4391709583685355, 0.8310744205423978]
[2, 2, 5, 2, 5, 2, 2, 2, 1, 5]
oracle_surp_estimates: spearmanr (-0.15951465589382513, 0.6598056698616883)
oracle_surp_estimates: pearsonr (-0.300218593235662, 0.3993312095811257)
each_personal_score: spearmanr (-0.12483755678647183, 0.7311340232557363)
each_personal_score: pearsonr (0.02775822251569579, 0.9393256533861113)
2
[0.4004501418761413, -0.6262960417323553, 1.0111129148797615, -0.1475

Function for calculating correlation per recipe

In [26]:
def corr_per_recipe(personalized_surp_estimates_recipelvl, users_surp_ratings_recipelvl):
	for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates_recipelvl, users_surp_ratings_recipelvl)):
		print user_idx
		print each_personal_score
		print each_users_rating
		print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
		print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

Mode: Personalized, per recipes, 95th percentile and max

In [27]:
corr_per_recipe(personalized_surp_estimates_recipelvl_95perc, users_surp_ratings_recipelvl)

0
[0.5825177510358344, 0.3766410246138465, 0.4655375735178752, 0.4897748608014796, 0.5902716631344865, -0.1015438469289178, 0.5902716631344865, -0.13292281789134483, 0.10722371741785079, 0.4335544132435427, 0.501867080242336, 0.46813092687529967, 0.49614863506720613, 0.30289717190641985, 0.2400077333819204, -0.32276945227510273, 0.5854322295209975, 0.13869065800962227, 0.028929721010827643, 0.5436363806717961, 0.00350265441727381, 0.10996219567635483, 0.1300521179116552, 0.09327731963588592, 0.03399429743285158, -0.1958527708008515, 0.12353876919099487, 0.09446871270303438, 0.5902716631344865, 0.4197450455767878, 0.2200140707333063, 0.2964162939295196, 0.4796247037922552, 0.2683081664936383, 0.12068708247559698, 0.41234407260111566, 0.18536242745123388, -0.10571924329097045, 0.4409490580672216, 0.14147283610290134, 0.3724233406871842, 0.5778892264175435, 0.17785771278205378, -0.031799987693967594, -0.22884928911851188, -0.07477927678320247, 0.027091592501152915, 0.5762016430116379, 0.2

In [28]:
corr_per_recipe(personalized_surp_estimates_recipelvl_max, users_surp_ratings_recipelvl)

0
[1.0420700709383728, 0.8217086324646126, 0.9499809219691805, 0.9531598300843317, 1.048121835880558, 0.4004501418761413, 1.048121835880558, 0.3531153558762443, 0.5631289475364123, 0.8764787875499727, 0.9674690575713439, 0.9330305118294296, 0.9472887412804829, 0.7542831926744691, 0.7383653378078759, 0.15243059451739382, 1.0450975402838405, 0.5690688686756857, 0.48790048028225025, 1.021739912141993, 0.44966594799104953, 0.5634013604948671, 0.5724651120690639, 0.559542059506078, 0.49291495555769343, 0.2742870569656812, 0.5948570833783334, 0.5488822103271285, 1.048121835880558, 0.9112649398540451, 0.7003652944750787, 0.760236127571921, 0.9516403010793356, 0.7246230570405177, 0.5963713040722619, 0.8631036844986874, 0.6666510301848653, 0.3497411820031651, 0.8844172075485195, 0.6045374508395838, 0.8052986051767145, 1.0383771510013051, 0.6711194619383872, 0.3929374208760536, 0.21813644804663868, 0.36429534563143423, 0.48527489739537805, 1.0447953248719428, 0.7246230570405177, 0.71852163136172

In [29]:
# 
corr_per_recipe(foodie_personalized_surp_estimates_recipelvl_95perc, foodie_surp_ratings_recipelvl)

0
[0.5825177510358344, 0.4897748608014796, 0.5902716631344865, 0.5902716631344865, 0.4335544132435427, 0.501867080242336, 0.46813092687529967, 0.49614863506720613, 0.2400077333819204, 0.5854322295209975, 0.5436363806717961, 0.5902716631344865, 0.2200140707333063, 0.4796247037922552, 0.2683081664936383, 0.41234407260111566, 0.4409490580672216, 0.3724233406871842, 0.5778892264175435, 0.5762016430116379, 0.2683081664936383, 0.4417200243783423, 0.24637627035046622, 0.25328962797699445, 0.007685279981911382, 0.48100553868153073, 0.2388921890299236, -0.10152953218954062, -0.36777006531195044, 0.2683081664936383, 0.47513919760182055, 0.2683081664936383, 0.4918499539813701, 0.2905442031736919, 0.17852841217002216, -0.018722888255688932, 0.2683081664936383, 0.27855181812333557, 0.22693329379339672]
[1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2]
each_personal_score: spearmanr (-0.12265539929868594, 0.45695105268675884)
each_p

In [30]:
corr_per_recipe(foodie_personalized_surp_estimates_recipelvl_max, foodie_surp_ratings_recipelvl)

0
[1.0420700709383728, 0.9531598300843317, 1.048121835880558, 1.048121835880558, 0.8764787875499727, 0.9674690575713439, 0.9330305118294296, 0.9472887412804829, 0.7383653378078759, 1.0450975402838405, 1.021739912141993, 1.048121835880558, 0.7003652944750787, 0.9516403010793356, 0.7246230570405177, 0.8631036844986874, 0.8844172075485195, 0.8052986051767145, 1.0383771510013051, 1.0447953248719428, 0.7246230570405177, 0.9017203980908947, 0.704823125897761, 0.712430265242932, 0.490875042549168, 0.9284391544399461, 0.6832254130132938, 0.3783977004728634, 0.10029284836538031, 0.7246230570405177, 0.938352422598457, 0.7246230570405177, 0.957661671399691, 0.7328973497670804, 0.6489103344782458, 0.4136486107692976, 0.7246230570405177, 0.7494108920241268, 0.6857476357882278]
[1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2]
each_personal_score: spearmanr (-0.1539264540703806, 0.34948580344230495)
each_personal_score: pearsonr (-0

In [31]:
# 
corr_per_recipe(novice_personalized_surp_estimates_recipelvl_95perc, novice_surp_ratings_recipelvl)

0
[0.3766410246138465, 0.4655375735178752, -0.1015438469289178, -0.13292281789134483, 0.10722371741785079, 0.30289717190641985, -0.32276945227510273, 0.13869065800962227, 0.028929721010827643, 0.00350265441727381, 0.10996219567635483, 0.1300521179116552, 0.09327731963588592, 0.03399429743285158, -0.1958527708008515, 0.12353876919099487, 0.09446871270303438, 0.4197450455767878, 0.2964162939295196, 0.12068708247559698, 0.18536242745123388, -0.10571924329097045, 0.14147283610290134, 0.17785771278205378, -0.031799987693967594, -0.22884928911851188, -0.07477927678320247, 0.027091592501152915, 0.26912608222124224, 0.14009325092390992, 0.147084321625455, 0.14063801990114144, 0.13123457560046353, 0.07424448182689283, -0.17143878318089456, -0.4706453226434828, -0.500289818616533, -0.1075435886149024, -0.031799987693967594, -0.3994153980904856, 0.11514059553724834, -1.7322226293830625, 0.2964162939295196, -0.2270906285344847, -1.2802713705836495, -0.20656095140996894, -0.1450291693077133]
[1, 2,

In [32]:
corr_per_recipe(novice_personalized_surp_estimates_recipelvl_max, novice_surp_ratings_recipelvl)

0
[0.8217086324646126, 0.9499809219691805, 0.4004501418761413, 0.3531153558762443, 0.5631289475364123, 0.7542831926744691, 0.15243059451739382, 0.5690688686756857, 0.48790048028225025, 0.44966594799104953, 0.5634013604948671, 0.5724651120690639, 0.559542059506078, 0.49291495555769343, 0.2742870569656812, 0.5948570833783334, 0.5488822103271285, 0.9112649398540451, 0.760236127571921, 0.5963713040722619, 0.6666510301848653, 0.3497411820031651, 0.6045374508395838, 0.6711194619383872, 0.3929374208760536, 0.21813644804663868, 0.36429534563143423, 0.48527489739537805, 0.7185216313617284, 0.5920639866085953, 0.6022290987172171, 0.5753534515678806, 0.5945107254379447, 0.5297502960793254, 0.2760169495862016, -0.07044176741913688, -0.002351258844356851, 0.37415244021656846, 0.3929374208760536, 0.09717144358406316, 0.5612865391584673, -1.2987107298720704, 0.760236127571921, 0.28328189976540574, -0.7089343014507407, 0.2640846535534821, 0.2927441208902306]
[1, 2, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 

Get averages for the recipe level arrays

In [33]:
# All users ratings
users_surp_ratings_recipelvl_avg = averageing_fn(users_surp_ratings_recipelvl)
# Foodies ratings
foodie_surp_ratings_recipelvl_avg = averageing_fn(foodie_surp_ratings_recipelvl)
# Novices ratings
novice_surp_ratings_recipelvl_avg = averageing_fn(novice_surp_ratings_recipelvl)
# All users estimates 95th perc
personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(personalized_surp_estimates_recipelvl_95perc)
# All users estimates max
personalized_surp_estimates_recipelvl_max_avg = averageing_fn(personalized_surp_estimates_recipelvl_max)
# Foodies estimates 95th perc
foodie_personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(foodie_personalized_surp_estimates_recipelvl_95perc)
# Foodies estimates max
foodie_personalized_surp_estimates_recipelvl_max_avg = averageing_fn(foodie_personalized_surp_estimates_recipelvl_max)
# Novices estimates 95th perc
novice_personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(novice_personalized_surp_estimates_recipelvl_95perc)
# Novices estimates max
novice_personalized_surp_estimates_recipelvl_max_avg = averageing_fn(novice_personalized_surp_estimates_recipelvl_max)

In [34]:
# Correlating personalized surprise estimates of 95th percentile with users' ratings
print 'spearmanr', spearmanr(personalized_surp_estimates_recipelvl_95perc_avg, users_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipelvl_95perc_avg, users_surp_ratings_recipelvl_avg)

spearmanr (0.6142709857998429, 0.05882959050940385)
pearsonr (0.5233605344579262, 0.12055640631026002)


In [35]:
# Correlating personalized surprise estimates of max with users' ratings
print 'spearmanr', spearmanr(personalized_surp_estimates_recipelvl_max_avg, users_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipelvl_max_avg, users_surp_ratings_recipelvl_avg)

spearmanr (0.5543421091364434, 0.0963317341031337)
pearsonr (0.45016934246668694, 0.19172150097817728)


In [36]:
# Correlating foodies personalized surprise estimates of 95th perc with users' ratings
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_recipelvl_95perc_avg, foodie_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_recipelvl_95perc_avg, foodie_surp_ratings_recipelvl_avg)

spearmanr (0.5132023220686198, 0.12925020431143283)
pearsonr (0.4071640551934986, 0.24287870766112735)


In [37]:
# Correlating foodies personalized surprise estimates of max with users' ratings
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_recipelvl_max_avg, foodie_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_recipelvl_max_avg, foodie_surp_ratings_recipelvl_avg)

spearmanr (0.5592589407158036, 0.09279882025376006)
pearsonr (0.5824084722675724, 0.07728073034412494)


In [38]:
# Correlating novices personalized surprise estimates of 95th perc with users' ratings
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_recipelvl_95perc_avg, novice_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_recipelvl_95perc_avg, novice_surp_ratings_recipelvl_avg)

spearmanr (0.7706746355884523, 0.009077964528829273)
pearsonr (0.8007621344313486, 0.005378611002689472)


In [39]:
# Correlating novices personalized surprise estimates of max with users' ratings
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_recipelvl_95perc_avg, novice_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_recipelvl_95perc_avg, novice_surp_ratings_recipelvl_avg)

spearmanr (0.7706746355884523, 0.009077964528829273)
pearsonr (0.8007621344313486, 0.005378611002689472)


In [40]:
# Correlating oracle estimates 95th perc with users' ratings
print 'spearmanr', spearmanr(oracle_surp_estimates_95perc, users_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(oracle_surp_estimates_95perc, users_surp_ratings_recipelvl_avg)

spearmanr (0.10487553416094876, 0.7730914705441726)
pearsonr (0.12168564363464526, 0.7377193010294987)


In [41]:
# Correlating oracle estimates max with users' ratings
print 'spearmanr', spearmanr(oracle_surp_estimates_max, users_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(oracle_surp_estimates_max, users_surp_ratings_recipelvl_avg)

spearmanr (0.2996443833169965, 0.40027788641899587)
pearsonr (0.18182959669553983, 0.615139427473129)


Modes: Personalized, all users, user flat, 95th percentile/max score


In [42]:
print '95th percentile'
print 'spearmanr', spearmanr(personalized_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'Max score'
print 'spearmanr', spearmanr(personalized_surp_estimates_userflat_max, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_userflat_max, users_surp_ratings_user_flat)

95th percentile
spearmanr (0.22489947014846723, 2.533717625526296e-11)
pearsonr (0.20695318880422547, 8.973296233763031e-10)
Max score
spearmanr (0.25633125778625643, 2.2743971325476138e-14)
pearsonr (0.24756977576379482, 1.7774061228784527e-13)


In [43]:
# Foodies
print '95th percentile'
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_userflat_95perc, foodie_surp_ratings_user_flat)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_userflat_95perc, foodie_surp_ratings_user_flat)
print 'Max score'
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_userflat_max, foodie_surp_ratings_user_flat)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_userflat_max, foodie_surp_ratings_user_flat)
# Novices
print '95th percentile'
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_userflat_95perc, novice_surp_ratings_user_flat)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_userflat_95perc, novice_surp_ratings_user_flat)
print 'Max score'
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_userflat_max, novice_surp_ratings_user_flat)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_userflat_max, novice_surp_ratings_user_flat)

95th percentile
spearmanr (0.22199346011475524, 9.631499867376071e-06)
pearsonr (0.20092152358951923, 6.44302397331895e-05)
Max score
spearmanr (0.2622014218409234, 1.4912311056019525e-07)
pearsonr (0.265563655510081, 1.0178127640672545e-07)
95th percentile
spearmanr (0.2532601668855727, 2.5924844588216206e-08)
pearsonr (0.24065832419351135, 1.2829110857906872e-07)
Max score
spearmanr (0.2777237440508387, 9.02345508628914e-10)
pearsonr (0.2559204595146284, 1.8291079410740014e-08)


Modes: Personalized, all recipes, recipe flat, 95th percentile/max score

In [44]:
print '95th percentile'
print 'spearmanr', spearmanr(personalized_surp_estimates_recipeflat_95perc, users_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipeflat_95perc, users_surp_ratings_recipe_flat)
print 'Max score'
print 'spearmanr', spearmanr(personalized_surp_estimates_recipeflat_max, users_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipeflat_max, users_surp_ratings_recipe_flat)


95th percentile
spearmanr (0.22489947014846723, 2.533717625526296e-11)
pearsonr (0.2069531888042255, 8.973296233763031e-10)
Max score
spearmanr (0.25633125778625643, 2.2743971325476138e-14)
pearsonr (0.24756977576379485, 1.7774061228784527e-13)


In [45]:
# Foodies
print '95th percentile'
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_recipeflat_95perc, foodie_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_recipeflat_95perc, foodie_surp_ratings_recipe_flat)
print 'Max score'
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_recipeflat_max, foodie_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_recipeflat_max, foodie_surp_ratings_recipe_flat)
# Novices
print '95th percentile'
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_recipeflat_95perc, novice_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_recipeflat_95perc, novice_surp_ratings_recipe_flat)
print 'Max score'
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_recipeflat_max, novice_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_recipeflat_max, novice_surp_ratings_recipe_flat)

95th percentile
spearmanr (0.22199346011475524, 9.631499867376071e-06)
pearsonr (0.20092152358951923, 6.44302397331895e-05)
Max score
spearmanr (0.2622014218409234, 1.4912311056019525e-07)
pearsonr (0.265563655510081, 1.0178127640672545e-07)
95th percentile
spearmanr (0.2532601668855727, 2.5924844588216206e-08)
pearsonr (0.24065832419351138, 1.2829110857906872e-07)
Max score
spearmanr (0.2777237440508387, 9.02345508628914e-10)
pearsonr (0.25592045951462833, 1.8291079410740014e-08)


Replicate the oracle by the number of the users to be able to correlate with the users' ratings (the arrays have to be the same size)

In [46]:
oracle_surp_estimates_repeated_95perc = [oracle_surp_estimates_95perc] * len(users_surp_ratings_userlvl)
oracle_surp_estimates_repeated_max = [oracle_surp_estimates_max] * len(users_surp_ratings_userlvl)
oracle_surp_estimates_userflat_95perc = [item for sublist in oracle_surp_estimates_repeated_95perc for item in sublist]
oracle_surp_estimates_userflat_max = [item for sublist in oracle_surp_estimates_repeated_max for item in sublist]

In [47]:
print '95th percentile'
print 'spearmanr', spearmanr(oracle_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(oracle_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'Max score'
print 'spearmanr', spearmanr(oracle_surp_estimates_userflat_max, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(oracle_surp_estimates_userflat_max, users_surp_ratings_user_flat)

95th percentile
spearmanr (0.037650674493277386, 0.2700620548249365)
pearsonr (0.041088435275697124, 0.22870352587937326)
Max score
spearmanr (0.10066169524054402, 0.0031247058077166225)
pearsonr (0.044049425321728605, 0.196866593866097)
